In [ ]:
#| default_exp main
import sys,os
from pathlib import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

# Principal
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [ ]:
#| export
import os
from pathlib import Path
import json
from typing import Union
from datetime import datetime

import pandas as pd
from fastcore.test import *
from rich import print
import pyodbc
from pymongo import MongoClient
from dotenv import load_dotenv, find_dotenv

from extracao.reading import read_base, read_aero
from extracao.format import merge_on_frequency, _filter_matlab, _format_matlab

load_dotenv(find_dotenv(), override=True)

True

In [ ]:
#| export
def get_db(
    path: Union[str, Path],  # Pasta onde salvar os arquivos",
    connSQL: pyodbc.Connection = None,  # Objeto de conexão do banco SQL Server
    clientMongoDB: MongoClient = None,  # Objeto de conexão do banco MongoDB
) -> pd.DataFrame:  # Retorna o DataFrame com as bases da Anatel e da Aeronáutica
    """Lê e opcionalmente atualiza as bases da Anatel, mescla as bases da Aeronáutica, salva e retorna o arquivo
    A atualização junto às bases de dados da Anatel é efetuada caso ambos objetos de banco `connSQL` e `clientMongoDB` forem válidos`
    """
    dest = Path(path)
    dest.mkdir(parents=True, exist_ok=True)
    print(":scroll:[green]Lendo as bases de dados da Anatel...")
    df = read_base(path) #, connSQL, clientMongoDB)
    df = _filter_matlab(df)
    mod_times = {"ANATEL": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
    print(":airplane:[blue]Requisitando os dados da Aeronáutica.")
    update = all([connSQL, clientMongoDB])
    aero = read_aero(path, update=update)
    mod_times["AERONAUTICA"] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print(":spoon:[yellow]Mesclando os dados da Aeronáutica.")
    df = _format_matlab(merge_on_frequency(df, aero))
    print(":card_file_box:[green]Salvando os arquivos...")
    df.to_parquet(f"{dest}/AnatelDB.parquet.gzip", compression="gzip", index=False)
    versiondb = json.loads((dest / "VersionFile.json").read_text())
    mod_times["ReleaseDate"] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    versiondb["anateldb"].update(mod_times)
    json.dump(versiondb, (dest / "VersionFile.json").open("w"))
    print("Sucesso :zap:")
    return df

In [ ]:
#|eval: false
import os
import warnings
from extracao.updates import connect_db
import numpy as np
from geopy.distance import geodesic


In [ ]:
#|eval: false
warnings.filterwarnings("ignore")

In [ ]:
#|eval: false
folder = Path.cwd().parent / 'dados'

In [ ]:
#|eval: false
conn = connect_db()
uri = os.environ['MONGO_URI']
mongo_client = MongoClient(uri)

{'version': '4.0.5',
 'gitVersion': '3739429dd92b92d1b0ab120911a23d50bf03c412',
 'targetMinOS': 'Windows 7/Windows Server 2008 R2',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 0, 5, 0],
 'openssl': {'running': 'Windows SChannel'},
 'buildEnvironment': {'distmod': '2008plus-ssl',
  'distarch': 'x86_64',
  'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'ccflags': '/nologo /EHsc /W3 /wd4355 /wd4800 /wd4267 /wd4244 /wd4290 /wd4068 /wd4351 /wd4373 /we4013 /we4099 /we4930 /WX /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /Zc:rvalueCast /Zc:strictStrings /volatile:iso /Gw /Gy /Zc:inline',
  'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'cxxflags': '/TP',
  'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF',
  'target_arch': 'x86_64',
  'target_os': 'windows'},
 'bits': 64,
 'debug': False,
 'maxBsonObjectSize': 16777216,
 '

In [ ]:
#|eval: false
# os.chdir(f"{Path.cwd().parent}/extracao/aero")
# df = get_db(folder)
df = get_db(folder) #, conn, mongo_client)
#df = pd.read_parquet(folder / 'AnatelDB.parquet.gzip')

📜Lendo as bases de dados da Anatel...

✈Requisitando os dados da Aeronáutica.

🥄Mesclando os dados da Aeronáutica.

🗃Salvando os arquivos...

Sucesso ⚡

In [ ]:
df[df.Description.str.contains('\[ICAO\]')]

,Id,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
38811,#38812,109.100,-25.600000,-54.466667,"[ICAO] ILS, FOZ DO IGUACU CATARATAS | [AISW] S...",-1,-1,NI,-1.0
38812,#38813,109.100,-25.600000,-54.466667,"[ICAO] ILS, FOZ DO IGUACU CATARATAS | [AISW] S...",108,696597080,A3E,6.0
38813,#38814,109.100,-21.787222,-43.386112,"[MOS] L, AL, Sinart - Sociedade Nacional De Ap...",108,696597080,A3E,6.0
38814,#38815,109.100,-21.783333,-43.383335,"[ICAO] ILS/DME, JUIZ DE FORA",-1,-1,NI,-1.0
38819,#38820,109.300,-25.533333,-49.166668,"[ICAO] ILS, CURITIBA AFONSO PEÑA | [AISW] SBCT...",-1,-1,NI,-1.0
...,...,...,...,...,...,...,...,...,...
98908,#98909,136.825,-8.126389,-34.922779,"[ICAO] AOC U 100/100, RECIFE",507,1009303802,A3E,6.0
98909,#98910,136.825,-8.126389,-34.922779,"[ICAO] AOC U 100/100, RECIFE",507,1009307174,A3E,6.0
98910,#98911,136.825,-8.122803,-34.919464,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009265412,A3E,6.0
98914,#98915,136.875,-22.909445,-43.167221,"[ICAO] AOC U 100/100, RIO DE JANEIRO - S. DUMONT",-1,-1,NI,-1.0


In [ ]:
aero = read_aero(folder)

In [ ]:
aero[aero.Description.str.contains(r'\[ICAO\] ILS, FOZ DO IGUACU CATARATAS')]

,Frequency,Latitude,Longitude,Description
40,109.1,-25.6,-54.46666717529297,"[ICAO] ILS, FOZ DO IGUACU CATARATAS"
2910,109.1,-25.6,-54.46666717529297,"[ICAO] ILS, FOZ DO IGUACU CATARATAS | [AISW] S..."


In [ ]:
aero = aero.drop_duplicates(subset=['Frequency', 'Latitude', 'Longitude'], keep='last', ignore_index=True)
aero

,Frequency,Latitude,Longitude,Description
0,115.4,-16.25,-49.0,"[ICAO] VOR/DME, ANAPOLIS"
1,112.0,-10.983334,-37.06666564941406,"[ICAO] VOR/DME, ARACAJU STA. MARIA"
2,113.3,-15.85,-52.38333511352539,"[ICAO] VOR/DME, BARRA DO GARCAS"
3,112.2,-22.35,-49.04999923706055,"[ICAO] VOR/DME, BAURU"
4,109.7,-19.65,-43.95000076293945,"[ICAO] ILS, BELO HORIZONTE TANCREDO NEVES"
...,...,...,...,...
3211,1090.0,2.22156,-55.94504928588867,[DOC] Radar secundário (Airbone SSR)
3212,1090.0,-5.53668,-47.460601806640625,[DOC] Radar secundário (Airbone SSR)
3213,1090.0,-6.64257,-51.95793151855469,[DOC] Radar secundário (Airbone SSR)
3214,1090.0,-11.61905,-50.69247055053711,[DOC] Radar secundário (Airbone SSR)


In [ ]:
aero[aero.Description.str.contains(r'\[ICAO\] ILS, FOZ DO IGUACU CATARATAS')]

,Frequency,Latitude,Longitude,Description
2415,109.1,-25.6,-54.46666717529297,"[ICAO] ILS, FOZ DO IGUACU CATARATAS | [AISW] S..."


In [ ]:
aero.to_parquet(folder / 'aero.parquet.gzip', compression='gzip')

In [ ]:
base = read_base(folder)
base[base.Número_Estação == '696597080']

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,...,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade,Município_IBGE,Latitude_IBGE,Longitude_IBGE,Coords_Valida_IBGE
665912,109.1,SINART - SOCIEDADE NACIONAL DE APOIO RODOVIARI...,50408840870,696597080,Juiz de Fora,3136702,MG,-21.787222222222166,-43.386111111111,AL,...,A3E,6.0,2034-02-13,L,MOS,1,Juiz de Fora,-21.760900000000000,-43.348800000000000,1


In [ ]:
aero

In [ ]:
#|eval: false
from pandas_profiling import ProfileReport
df['Frequency'] = df['Frequency'].astype('category')
profile = ProfileReport(df, config_file= folder.parent / 'nbs/report_config.yaml')
profile.to_notebook_iframe()